### Cover Image Finder

I will be utilizing Open Library and possibly Google Books API to locate the ISBN, Cover Art, and # of Pages for all of my materials.

In [1]:
%pip install jupyterlab pandas sqlalchemy psycopg2-binary requests beautifulsoup4 python-dotenv rich pathlib

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import requests
import time
from pathlib import Path
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
from dotenv import load_dotenv
import pandas as pd
from rich import print
from rich.progress import track
print("[bold green]Environment setup complete.[/bold green]")

Environment setup complete.

In [3]:
env_path = Path('.').resolve().parent / '.env'
load_dotenv(dotenv_path=env_path)

DATABASE_URL = os.getenv('DATABASE_URL')
if DATABASE_URL and DATABASE_URL.startswith("postgresql+asyncpg://"):
    DATABASE_URL = DATABASE_URL.replace("postgresql+asyncpg://", "postgresql://")
    print("[yellow]Switched to synchronous 'postgresql://' driver.[/yellow]")

if not DATABASE_URL:
    print("[bold red]Error: DATABASE_URL not set.[/bold red]")
else:
    print("[bold green]Database URL loaded.[/bold green]")
    print("[bold blue]Connecting to database...[/bold blue]")

Database URL loaded.

Connecting to database...

In [4]:
from sqlalchemy.exc import OperationalError

engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()

print("[bold green]Database connection established.\nAttempting DB connection test...[/bold green]")

try:
    session.execute(text("SELECT 1"))
    print("[bold green]Database connection test successful![/bold green]")

except OperationalError as e:
    print(f"[bold red]Database connection test failed.\nCheck DATABASE URL.\nError details: {e}[/bold red]")
    session.close()

print("[bold green]Database session created, let's proceed.[/bold green]")

Database connection established.
Attempting DB connection test...

Database connection test successful!

Database session created, let's proceed.

In [5]:
from rich import print
from rich.table import Table
from rich.console import Console

# 1. Get the ENTIRE material table in one go
print("Fetching all materials from database...")
try:
    df_all = pd.read_sql("SELECT material_id, title, isbn, cover_image_url, page_count FROM material", engine)

    # 2. Filter the DataFrame in Pandas to find what needs updating
    df_to_update = df_all[
        df_all['isbn'].isnull() | 
        df_all['cover_image_url'].isnull() | 
        df_all['page_count'].isnull()
    ]
    
    # --- 3. Get your counts ---
    total_materials = len(df_all)
    found_to_update = len(df_to_update)

    if found_to_update == 0:
        print("No materials need updating.")
    else:
        # --- 4. Your updated print statement ---
        print(f"Found [bold green]{found_to_update}[/bold green] / [bold]{total_materials}[/bold] materials to update. Showing first 5:")
        
        # --- Rich Table Logic (now uses df_to_update) ---
        console = Console()
        table = Table(show_header=True, header_style="bold magenta")
        table.add_column("material_id", style="dim", width=12)
        table.add_column("title", style="cyan")
        
        for index, row in df_to_update.head().iterrows():
            table.add_row(
                str(row['material_id']), 
                row['title']
            )
            
        console.print(table)
        # --- End Rich Table Logic ---

except Exception as e:
    print(f"[bold red]Error fetching data:[/bold red] {e}")
print("[bold green]Data fetch and filtering complete.[/bold green]")

Fetching all materials from database...

Found 57 / 57 materials to update. Showing first 5:

┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ material_id  ┃ title                                                                                            ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1            │ AI Engineering: Building Applications with Foundation Models                                     │
│ 2            │ An Introduction to Python: A Guide for Marketers, Developers, and Data Analysts                  │
│ 3            │ An Introduction to Statistical Learning: Applications in Python                                  │
│ 4            │ Applied Machine Learning and AI for Engineers: Solve Business Problems That Can't Be Solved      │
│              │ Algorithmically                                                                                  │
│ 5            │ Artificial Intelligence: A Modern Approach                                                       │
└──────────────┴──────────────────────────────────────────────────────────────────────────────────────────────────┘

Data fetch and filtering complete.

In [6]:
# --- API Endpoints ---
SEARCH_API_URL = "https://openlibrary.org/search.json"
COVER_API_URL = "https://covers.openlibrary.org/b/id/"

results_to_update = []

print(f"\nStarting API enrichment for {len(df_to_update)} materials...")

for index, row in df_to_update.iterrows():
    material_id = row['material_id']
    title = row['title']
    
    print(f"\nSearching for: [bold]{title}[/bold]...")
    
    try:
        # --- 1. Use the Search API by title ---
        search_params = {"title": title, "limit": 1}
        response = requests.get(SEARCH_API_URL, params=search_params, timeout=10)
        response.raise_for_status() 
        
        search_data = response.json()
        
        if search_data.get('numFound', 0) > 0:
            top_result = search_data['docs'][0]
            
            # --- 2. Extract all three pieces of data ---
            
            # Get ISBN (it's an array, we'll just take the first one)
            isbn = top_result.get('isbn', [None])[0]
            
            # Get Page Count
            page_count = top_result.get('number_of_pages_median')
            
            # Get Cover URL
            cover_url = None
            cover_id = top_result.get('cover_i')
            if cover_id:
                cover_url = f"{COVER_API_URL}{cover_id}-L.jpg" # -L for Large
            
            print(f"  ISBN: {isbn}")
            print(f"  Page_Count: {page_count}")
            print(f"  Cover: {cover_url}")
            
            # Store the results
            results_to_update.append({
                'material_id': material_id,
                'isbn': isbn,
                'page_count': page_count,
                'cover_image_url': cover_url
            })
            
        else:
            print(f"No results found for '{title}'.")

        time.sleep(1) 
        
    except requests.exceptions.RequestException as e:
        print(f"Error searching for '{title}': {e}")

print(f"\nFinished API calls. Found data for {len(results_to_update)} materials.")

Starting API enrichment for 57 materials...

Searching for: AI Engineering: Building Applications with Foundation Models...

No results found for 'AI Engineering: Building Applications with Foundation Models'.

Searching for: An Introduction to Python: A Guide for Marketers, Developers, and Data Analysts...

No results found for 'An Introduction to Python: A Guide for Marketers, Developers, and Data Analysts'.

...

Searching for: Weapons of Math Destruction...

ISBN: None

Page_Count: None

Cover: https://covers.openlibrary.org/b/id/8136557-L.jpg

Finished API calls. Found data for 19 materials.

In [11]:
from rich import print
from rich.table import Table
from rich.console import Console

# --- Initialize Counts ---
isbn_count = 0
pages_count = 0
cover_count = 0
complete_count = 0
total_found = len(results_to_update)

# --- Rich Table Setup ---
console = Console()
summary_table = Table(
    title=f"Analysis of {total_found} Found Materials",
    show_header=True, 
    header_style="bold magenta"
)
summary_table.add_column("ID", style="dim", width=8, justify="right")
summary_table.add_column("Title (from DB)", style="cyan")
summary_table.add_column("ISBN Found?", justify="center")
summary_table.add_column("Pages Found?", justify="center")
summary_table.add_column("Cover Found?", justify="center")

print(f"\n--- Analyzing {total_found} Results from Open Library ---")

found_ids = set()

# --- Loop 1: Analyze results and build table ---
for item in results_to_update:
    material_id = item['material_id']
    found_ids.add(material_id) 
    
    title = df_to_update[df_to_update['material_id'] == material_id]['title'].values[0]
    
    # --- THIS IS THE FIX ---
    # Replace emojis with colored text
    isbn_found = "[green]Yes[/green]" if item['isbn'] else "[red]No[/red]"
    pages_found = "[green]Yes[/green]" if item['page_count'] else "[red]No[/red]"
    cover_found = "[green]Yes[/green]" if item['cover_image_url'] else "[red]No[/red]"
    # --- END FIX ---
    
    summary_table.add_row(
        str(material_id), title, isbn_found, pages_found, cover_found
    )

    # --- Tally Counts (unchanged) ---
    is_complete = True
    if item['isbn']:
        isbn_count += 1
    else:
        is_complete = False
        
    if item['page_count']:
        pages_count += 1
    else:
        is_complete = False
        
    if item['cover_image_url']:
        cover_count += 1
    else:
        is_complete = False
        
    if is_complete:
        complete_count += 1

# --- Print the Results ---
console.print(summary_table)

print("\n--- [bold]Open Library Summary[/bold] ---")
print(f"Materials with new ISBN:       [bold green]{isbn_count}[/bold green] / {total_found}")
print(f"Materials with new Page Count: [bold green]{pages_count}[/bold green] / {total_found}")
print(f"Materials with new Cover URL:  [bold green]{cover_count}[/bold green] / {total_found}")
print(f"Materials 'Completed' (all 3): [bold blue]{complete_count}[/bold blue] / {total_found}")


# --- BUILD AND PRINT THE SECOND TABLE (FOUND vs. NOT FOUND) ---
print("\n" + "—"*80)
print("\n--- [bold]Material API Lookup Status[/bold] ---")

found_table = Table(title="[bold green]Materials Found in API[/bold green]", show_header=True, header_style="green", min_width=50)
found_table.add_column("ID", style="dim", width=8, justify="right")
found_table.add_column("Title")

not_found_table = Table(title="[bold red]Materials Not Found in API[/bold red]", show_header=True, header_style="red", min_width=50)
not_found_table.add_column("ID", style="dim", width=8, justify="right")
not_found_table.add_column("Title")

# Loop through the *original* df_to_update to sort all 57 items
for index, row in df_to_update.iterrows():
    material_id = row['material_id']
    title = row['title']
    
    if material_id in found_ids:
        found_table.add_row(str(material_id), title)
    else:
        not_found_table.add_row(str(material_id), title)

console.print(found_table)
console.print(not_found_table)

--- Analyzing 19 Results from Open Library ---

                                          Analysis of 19 Found Materials                                           
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃       ID ┃ Title (from DB)                                          ┃ ISBN Found? ┃ Pages Found? ┃ Cover Found? ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│        5 │ Artificial Intelligence: A Modern Approach               │     No      │      No      │     Yes      │
│        7 │ Big Book of Data Science Use Cases                       │     No      │      No      │     Yes      │
│        9 │ Build a Career in Data Science                           │     No      │      No      │      No      │
│       12 │ Data Science for Dummies                                 │     No      │      No      │     Yes      │
│       13 │ Data Science from Scratch                                │     No      │      No      │     Yes      │
│       14 │ Deep Learning                                            │     No      │      No      │     Yes      │
│       21 │ Feature Engineering for Machine Learning: Principles and │     No      │      No      │     Yes      │
│          │ Techniques for Data Scientists                           │             │              │              │
│       27 │ Hundred-Page Language Models Book                        │     No      │      No      │     Yes      │
│       28 │ Impractical Python Projects: Playful Programming         │     No      │      No      │     Yes      │
│          │ Activities to Make You Smarter                           │             │              │              │
│       30 │ Learning Spark: Lightning-Fast Data Analysis             │     No      │      No      │     Yes      │
│       34 │ Machine Learning for Humans                              │     No      │      No      │     Yes      │
│       36 │ Mathematics for Machine learning:                        │     No      │      No      │     Yes      │
│       38 │ Pattern Recognition and Machine Learning                 │     No      │      No      │     Yes      │
│       43 │ Probability and Statistics for Computer Scientists       │     No      │      No      │     Yes      │
│       46 │ Python for Data Science Handbook                         │     No      │      No      │      No      │
│       48 │ R for Data Science                                       │     No      │      No      │      No      │
│       50 │ Spark: The Definitive Guide: Big Data Processing Made    │     No      │      No      │     Yes      │
│          │ Simple                                                   │             │              │              │
│       52 │ The Art of Data Science                                  │     No      │      No      │      No      │
│       57 │ Weapons of Math Destruction                              │     No      │      No      │     Yes      │
└──────────┴──────────────────────────────────────────────────────────┴─────────────┴──────────────┴──────────────┘

--- Open Library Summary ---

Materials with new ISBN:       0 / 19

Materials with new Page Count: 0 / 19

...

Materials 'Completed' (all 3): 0 / 19

————————————————————————————————————————————————————————————————————————————————

--- Material API Lookup Status ---

                                        Materials Found in API                                        
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃       ID ┃ Title                                                                                   ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        5 │ Artificial Intelligence: A Modern Approach                                              │
│        7 │ Big Book of Data Science Use Cases                                                      │
│        9 │ Build a Career in Data Science                                                          │
│       12 │ Data Science for Dummies                                                                │
│       13 │ Data Science from Scratch                                                               │
│       14 │ Deep Learning                                                                           │
│       21 │ Feature Engineering for Machine Learning: Principles and Techniques for Data Scientists │
│       27 │ Hundred-Page Language Models Book                                                       │
│       28 │ Impractical Python Projects: Playful Programming Activities to Make You Smarter         │
│       30 │ Learning Spark: Lightning-Fast Data Analysis                                            │
│       34 │ Machine Learning for Humans                                                             │
│       36 │ Mathematics for Machine learning:                                                       │
│       38 │ Pattern Recognition and Machine Learning                                                │
│       43 │ Probability and Statistics for Computer Scientists                                      │
│       46 │ Python for Data Science Handbook                                                        │
│       48 │ R for Data Science                                                                      │
│       50 │ Spark: The Definitive Guide: Big Data Processing Made Simple                            │
│       52 │ The Art of Data Science                                                                 │
│       57 │ Weapons of Math Destruction                                                             │
└──────────┴─────────────────────────────────────────────────────────────────────────────────────────┘

                                            Materials Not Found in API                                             
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃       ID ┃ Title                                                                                                ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        1 │ AI Engineering: Building Applications with Foundation Models                                         │
│        2 │ An Introduction to Python: A Guide for Marketers, Developers, and Data Analysts                      │
│        3 │ An Introduction to Statistical Learning: Applications in Python                                      │
│        4 │ Applied Machine Learning and AI for Engineers: Solve Business Problems That Can't Be Solved          │
│          │ Algorithmically                                                                                      │
│        6 │ Big Book of Data Engineering                                                                         │
│        8 │ Big Book of Machine Learning Use Cases: A Collection of Technical Blogs, Including Code Samples and  │
│          │ Notebooks                                                                                            │
│       10 │ Building Machine Learning Powered Applications: Going from Idea to Product                           │
│       11 │ Computer Vision: Algorithms and Applications                                                         │
│       15 │ Deep Learning for Finance: Creating Machine & Deep Learning Models for Trading in Python             │
│       16 │ Designing Machine Learning Systems: An Iterative Process for Production-Ready Applications           │
│       17 │ Digital Image Processing (EEC 022)                                                                   │
│       18 │ Digital Image Processing : CSC 4260 Lecture                                                          │
│       19 │ Effective Python: 125 Specific Ways to Write Better Python                                           │
│       20 │ Everybody Lies: Big Data, New Data, and What the Internet Can Tell Us About Who We Really Are        │
│       22 │ GANs in Action: Deep Learning with Generative Adversarial Networks                                   │
│       23 │ Generative Deep Learning: Teaching Machines to Paint, Write, Compose, and Play                       │
│       24 │ Hands-On Generative AI with Transformers and Diffusion Models: (Early Release)                       │
│       25 │ Hands-On Large Language Models: Language Understanding and Generation                                │
│       26 │ Hands-On Machine Learning with Scikit-Learn, Keras & TensorFlow: Concepts, Tools, and Techniques to  │
│          │ Build Intelligent Systems                                                                            │
│       29 │ Introduction to ML with Python                                                                       │
│       31 │ LLM Engineer's Handbook: Master the Art of Engineering Large Language Models from Concept to         │
│          │ Production                                                                                           │
│       32 │ Machine Learning for Absolute Beginners: A Plain English Introduction                                │
│       33 │ Machine Learning for Hackers: Case Studies and Algorithms to Get You Started                         │
│       35 │ Machine Learning: A Probabilistic Perspective                                                        │
│       37 │ Natural Language Processing with Transformers: Building Language Applications with Hugging Face      │
│       39 │ Practical MLOps: Operationalizing Machine Learning Models                                            │
│       40 │ Practical Statistics for Data Scientists: 5

In [16]:
session.close()
print("\n[bold green]Database session closed. Script complete.[/bold green]")
print("Your next step will be to update the database and pull from Google Books API.")

Database session closed. Script complete.

Your next step will be to update the database and pull from Google Books API.